# Imports

In [1]:
import ee
import numpy as np
import math
import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
#ee.Authenticate()


In [3]:

# Initialize Google Earth Engine
ee.Initialize()

In [4]:
# Load study area and community boundaries
nm_path = r"C:\Users\bsf31\Documents\data\NM\nm_vector.gpkg"
study_boundary = gpd.read_file(nm_path, layer='counties_dissolved')

santa_clara_gdf = gpd.read_file(nm_path, layer='santa_clara_small')


In [5]:
# Convert to Earth Engine geometry
ee_boundary = geemap.geopandas_to_ee(study_boundary)

ee_santa_clara = geemap.geopandas_to_ee(santa_clara_gdf).geometry()


In [6]:
ee_boundary_coordinates = ee_boundary.geometry()

# Landsat Collection 2

In [7]:
# Function to scale the surface temperature band and convert to Fahrenheit
def scale_lst(image):
    lst_k = image.select('ST_B10').multiply(0.00341802).add(149.0)
    lst_f = lst_k.subtract(273.15).multiply(1.8).add(32)
    return image.addBands(lst_f.rename('LST_F'))

# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4'])
    return image.addBands(ndvi.rename('NDVI'))

# Function to calculate Fraction of Vegetation Cover
def calculate_fvc(image):
    fvc = image.select('NDVI').subtract(0.2).divide(0.3).pow(2).clamp(0, 1)
    return image.addBands(fvc.rename('FVC'))

# Function to calculate Emissivity
def calculate_emissivity(image):
    emissivity = image.select('FVC').multiply(0.0004).add(0.986)
    return image.addBands(emissivity.rename('Emissivity'))

# Define a function to clip each image to the ROI
def clip_to_region(image,roi):
    return image.clip(roi)


In [8]:
pt = ee.Geometry.Point([-108.08, 32.46])   # Point corresponding to Santa Barbara, CA


# Load the Landsat 8 collection and filter images based on criteria
landsat_collection1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2019-04-01', '2024-11-01') \
    .filter(ee.Filter.calendarRange(5, 9, 'month')) \
    .filterBounds(pt)\
    .filter(ee.Filter.lt('CLOUD_COVER', 9))  # Filter for images with less than 10% cloud cover



In [9]:
""" pt = ee.Geometry.Point([-108.08, 32.46])   # Point corresponding to Santa Barbara, CA


# Load the Landsat 8 collection and filter images based on criteria
landsat_collection1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2019-04-01', '2019-09-18') \
    .filter(ee.Filter.calendarRange(5, 9, 'month')) \
    .filterBounds(pt)\
    .filter(ee.Filter.lt('CLOUD_COVER', 9))  # Filter for images with less than 10% cloud cover
# Load the Landsat 8 collection and filter images based on criteria
landsat_collection2 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate('2019-09-19', '2024-08-14') \
    .filter(ee.Filter.calendarRange(5, 9, 'month')) \
    .filterBounds(pt)\
    .filter(ee.Filter.lt('CLOUD_COVER', 9))  # Filter for images with less than 10% cloud cover

# Load the Landsat 8 collection and filter images based on criteria
landsat_collection3 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterDate( '2024-08-15', '2024-11-01') \
    .filter(ee.Filter.calendarRange(5, 9, 'month')) \
    .filterBounds(pt)\
    .filter(ee.Filter.lt('CLOUD_COVER', 9))  # Filter for images with less than 10% cloud cover """

' pt = ee.Geometry.Point([-108.08, 32.46])   # Point corresponding to Santa Barbara, CA\n\n\n# Load the Landsat 8 collection and filter images based on criteria\nlandsat_collection1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")     .filterDate(\'2019-04-01\', \'2019-09-18\')     .filter(ee.Filter.calendarRange(5, 9, \'month\'))     .filterBounds(pt)    .filter(ee.Filter.lt(\'CLOUD_COVER\', 9))  # Filter for images with less than 10% cloud cover\n# Load the Landsat 8 collection and filter images based on criteria\nlandsat_collection2 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")     .filterDate(\'2019-09-19\', \'2024-08-14\')     .filter(ee.Filter.calendarRange(5, 9, \'month\'))     .filterBounds(pt)    .filter(ee.Filter.lt(\'CLOUD_COVER\', 9))  # Filter for images with less than 10% cloud cover\n\n# Load the Landsat 8 collection and filter images based on criteria\nlandsat_collection3 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")     .filterDate( \'2024-08-15\', \'2024-11-01\')     

In [10]:
""" # Combine the collections
combined_collection = landsat_collection1.merge(landsat_collection2).merge(landsat_collection3)
# Get the list of image dates that meet the criteria
image_dates = combined_collection.aggregate_array('system:time_start').map(lambda t: ee.Date(t).format('YYYY-MM-dd')).getInfo()

print("Selected Image Dates:")
print(image_dates)
combined_collection
# Apply all processing steps
processed_collection = combined_collection \
    .map(lambda image: image.clip(ee_hurley)) \
    .map(scale_lst) \
    .map(calculate_ndvi) \
    .map(calculate_fvc) \
    .map(calculate_emissivity) 
 """

' # Combine the collections\ncombined_collection = landsat_collection1.merge(landsat_collection2).merge(landsat_collection3)\n# Get the list of image dates that meet the criteria\nimage_dates = combined_collection.aggregate_array(\'system:time_start\').map(lambda t: ee.Date(t).format(\'YYYY-MM-dd\')).getInfo()\n\nprint("Selected Image Dates:")\nprint(image_dates)\ncombined_collection\n# Apply all processing steps\nprocessed_collection = combined_collection     .map(lambda image: image.clip(ee_hurley))     .map(scale_lst)     .map(calculate_ndvi)     .map(calculate_fvc)     .map(calculate_emissivity) \n '

In [11]:
# Get the list of image dates that meet the criteria
image_dates = landsat_collection1.aggregate_array('system:time_start').map(lambda t: ee.Date(t).format('YYYY-MM-dd')).getInfo()

print("Selected Image Dates:")
print(image_dates)



Selected Image Dates:
['2019-05-29', '2019-06-30', '2019-08-01', '2019-09-02', '2019-09-18', '2020-05-15', '2020-05-31', '2020-06-16', '2020-08-03', '2020-08-19', '2020-09-04', '2020-09-20', '2021-05-02', '2021-06-03', '2021-06-19', '2021-07-21', '2021-08-06', '2021-08-22', '2021-09-07', '2022-05-05', '2022-05-21', '2022-06-06', '2022-07-08', '2022-09-10', '2023-05-08', '2023-05-24', '2023-06-25', '2023-07-27', '2023-09-29', '2024-05-10', '2024-05-26', '2024-06-11', '2024-07-13', '2024-08-14', '2024-08-30', '2019-05-13', '2019-05-29', '2019-06-30', '2019-07-16', '2019-08-01', '2019-08-17', '2019-09-02', '2019-09-18', '2020-05-15', '2020-05-31', '2020-06-16', '2020-08-03', '2020-08-19', '2020-09-04', '2020-09-20', '2021-05-18', '2021-06-19', '2021-07-21', '2021-08-06', '2021-08-22', '2021-09-07', '2022-05-05', '2022-05-21', '2022-06-06', '2022-07-08', '2022-07-24', '2022-08-09', '2022-09-10', '2023-05-08', '2023-05-24', '2023-06-25', '2023-07-11', '2023-07-27', '2023-08-12', '2023-09-29

In [12]:

# Apply all processing steps
processed_collection = landsat_collection1 \
    .map(lambda image: image.clip(ee_santa_clara)) \
    .map(scale_lst) \
    .map(calculate_ndvi) \
    .map(calculate_fvc) \
    .map(calculate_emissivity) 

In [13]:
processed_collection

In [14]:
# Function to extract LST values
def extract_lst(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mean_lst = image.select('LST_F').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee_santa_clara,
        scale=30
    ).get('LST_F')
    return ee.Feature(None, {'date': date, 'LST': mean_lst})

# Extract LST values
lst_values = processed_collection.map(extract_lst)


In [17]:
lst_values.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'version': 1738180792555029,
 'id': 'LANDSAT/LC08/C02/T1_L2',
 'properties': {'type_name': 'ImageCollection',
  'keywords': ['cfmask',
   'cloud',
   'fmask',
   'global',
   'l8sr',
   'landsat',
   'lasrc',
   'lst',
   'reflectance',
   'sr',
   'usgs'],
  'visualization_1_bands': 'SR_B5,SR_B4,SR_B3',
  'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png',
  'visualization_1_max': '30000.0',
  'description': '<p>This dataset contains atmospherically corrected\nsurface reflectance and land surface temperature derived from the data\nproduced by the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and one thermal infrared (TIR) band processed to orthorectified\nsurface temperature. They also contain intermediate bands used in\ncalculation of the ST products, as well as QA bands.</p><p>Landsat 8

In [ ]:
# Print results
print("Date, Mean LST (°F)")
lst_list = lst_values.getInfo().get('features')
for feature in lst_list:
    date = feature['properties']['date']
    lst = feature['properties']['LST']
    print(f"{date}, {lst:.2f}")

In [28]:


# Print collection information
print(f"\nNumber of images: {processed_collection.size().getInfo()}")
#print(f"Date range: {start_date} to {end_date}")

# Print cloud cover information
cloud_covers = processed_collection.aggregate_array('CLOUD_COVER').getInfo()
dates = processed_collection.aggregate_array('system:time_start').getInfo()
dates = [ee.Date(d).format('YYYY-MM-dd').getInfo() for d in dates]

print("\nCloud cover for each image:")
for date, cloud_cover in zip(dates, cloud_covers):
    print(f"{date}: {cloud_cover:.2f}%")


Number of images: 77

Cloud cover for each image:
2019-05-29: 7.42%
2019-06-30: 7.17%
2019-08-01: 7.59%
2019-09-02: 4.46%
2019-09-18: 7.68%
2020-05-15: 0.03%
2020-05-31: 2.03%
2020-06-16: 4.03%
2020-08-03: 0.02%
2020-08-19: 0.34%
2020-09-04: 0.61%
2020-09-20: 0.43%
2021-05-02: 0.06%
2021-06-03: 5.08%
2021-06-19: 0.91%
2021-07-21: 7.28%
2021-08-06: 4.96%
2021-08-22: 7.42%
2021-09-07: 0.00%
2022-05-05: 0.01%
2022-05-21: 0.06%
2022-06-06: 0.01%
2022-07-08: 3.63%
2022-09-10: 3.26%
2023-05-08: 0.02%
2023-05-24: 0.14%
2023-06-25: 0.04%
2023-07-27: 3.43%
2023-09-29: 0.24%
2024-05-10: 8.20%
2024-05-26: 0.06%
2024-06-11: 3.74%
2024-07-13: 0.65%
2024-08-14: 8.54%
2024-08-30: 7.45%
2019-05-13: 7.73%
2019-05-29: 0.00%
2019-06-30: 4.28%
2019-07-16: 5.53%
2019-08-01: 3.76%
2019-08-17: 6.91%
2019-09-02: 0.69%
2019-09-18: 3.31%
2020-05-15: 0.00%
2020-05-31: 0.01%
2020-06-16: 0.34%
2020-08-03: 1.24%
2020-08-19: 0.00%
2020-09-04: 0.05%
2020-09-20: 0.58%
2021-05-18: 6.77%
2021-06-19: 0.00%
2021-07-21: 8

In [ ]:
""" ## Define the specific dates you want to inspect
specific_dates = ['2019-09-18', '2024-08-14']  # Replace with your desired dates

# Filter the collection for specific dates using ee.Filter.Or
filtered_collection = processed_collection.filter(
    ee.Filter.Or(
        ee.Filter.date(specific_dates[0], ee.Date(specific_dates[0]).advance(1, 'day')),
        ee.Filter.date(specific_dates[1], ee.Date(specific_dates[1]).advance(1, 'day'))
    )
)

# Check if the collection is empty
if filtered_collection.size().getInfo() == 0:
    raise ValueError("No images found for the specified dates.")

# Create the map and add true-color layers for the specific dates
Map = geemap.Map()
Map.center_object(ee_hurley, 12)

# Visualization parameters for true-color composite
true_color_vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # Red, Green, Blue
    'min': 0,
    'max': 3000,  # Adjust based on the reflectance range of your data
    'gamma': 1.4  # Optional: adjust for better visualization
}

# Add true-color layers for each date
filtered_images = filtered_collection.toList(filtered_collection.size())
for i in range(filtered_collection.size().getInfo()):
    image = ee.Image(filtered_images.get(i))
    image_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    Map.addLayer(image, true_color_vis_params, f'True Color {image_date}')

Map.addLayer(ee_hurley, {}, 'Hurley Boundary')
Map """




In [18]:
# Calculate the mean image across the entire collection
collection_mean = processed_collection.mean()

# Extract the mean LST over the Hurley area
mean_lst = collection_mean.select('LST_F').reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ee_santa_clara,
    scale=30,
    maxPixels=1e8,
    bestEffort=True
).get('LST_F')

# Get the result as a Python object
mean_lst_value = mean_lst.getInfo()

print("Mean LST (°F) for the entire collection over Hurley, NM:", mean_lst_value)


Mean LST (°F) for the entire collection over Hurley, NM: 113.88991089271732


In [19]:
# Set up export parameters
export_task = ee.batch.Export.image.toDrive(
    image=collection_mean.select('LST_F'),  # Select the LST band
    description='Hurley_LST_Mean',
    folder='lst',  # Optional: specify a folder in Google Drive
    fileNamePrefix='Hurley_LST_Mean_Image',
    region=ee_santa_clara,
    scale=30,
    crs='EPSG:4326',  # 'EPSG:32612',  # Use UTM Zone 12N to match the Hurley vectors
    maxPixels=1e13
)

# Start the export task
export_task.start()
print("Exporting mean LST image to Google Drive...")

Exporting mean LST image to Google Drive...


In [20]:
Map = geemap.Map()
Map.center_object(ee_santa_clara, 12)
Map.addLayer(collection_mean, {}, 'collection_mean')
Map


Map(center=[32.775045669116146, -108.15246441481047], controls=(WidgetControl(options=['position', 'transparen…

In [23]:
# Function to calculate per-image statistics for LST
def calculate_stats_per_image(image):
    stats = image.select('LST_F').reduceRegion(
        reducer=ee.Reducer.mean()
            .combine(ee.Reducer.min(), sharedInputs=True)
            .combine(ee.Reducer.max(), sharedInputs=True)
            .combine(ee.Reducer.stdDev(), sharedInputs=True),
        geometry=ee_santa_clara,
        scale=30,
        maxPixels=1e8,
        bestEffort=True
    )
    return image.set(stats)

In [24]:
# Apply the function to each image in the collection to get per-image statistics
stats_per_image = processed_collection.map(calculate_stats_per_image)

In [25]:
stats_per_image

In [26]:
# Extract individual statistics
dates = stats_per_image.aggregate_array('system:time_start').getInfo()
means = stats_per_image.aggregate_array('LST_F_mean').getInfo()
mins = stats_per_image.aggregate_array('LST_F_min').getInfo()
maxs = stats_per_image.aggregate_array('LST_F_max').getInfo()
std_devs = stats_per_image.aggregate_array('LST_F_stdDev').getInfo()


In [27]:
# Print per-image statistics
print("Per-image LST statistics:")
for date, mean, min_, max_, std_dev in zip(dates, means, mins, maxs, std_devs):
    formatted_date = ee.Date(date).format("YYYY-MM-dd").getInfo()
    print(f"Date: {formatted_date}, Mean LST: {mean}, Min LST: {min_}, Max LST: {max_}, StdDev LST: {std_dev}")

Per-image LST statistics:
Date: 2019-05-29, Mean LST: 110.93076781122572, Min LST: 100.55537474000003, Max LST: 126.84473376800001, StdDev LST: 4.032622865020221
Date: 2019-06-30, Mean LST: 118.61436754504007, Min LST: 110.07934566800004, Max LST: 129.62563484000003, StdDev LST: 3.29092759878444
Date: 2019-08-01, Mean LST: 111.93300976482459, Min LST: 103.10863568000003, Max LST: 119.15418876800007, StdDev LST: 3.070540799444852
Date: 2019-09-02, Mean LST: 117.31671645108644, Min LST: 107.74757242400001, Max LST: 129.60102509600006, StdDev LST: 3.5685096476914686
Date: 2019-09-18, Mean LST: 53.648046481521455, Min LST: 34.619718128000045, Max LST: 103.56391594399997, StdDev LST: 14.332257656658705
Date: 2020-05-15, Mean LST: 112.47757665631468, Min LST: 102.24729464000002, Max LST: 121.75666919600005, StdDev LST: 3.2418929397763168
Date: 2020-05-31, Mean LST: 117.16838000220673, Min LST: 107.68604806400005, Max LST: 125.31892964000008, StdDev LST: 2.9728206724760784
Date: 2020-06-16, M

In [30]:
# Calculate overall statistics for the entire collection (min, max, stdDev)
collection_min = processed_collection.select('LST_F').reduce(ee.Reducer.min()).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ee_santa_clara,
    scale=30,
    maxPixels=1e8,
    bestEffort=True
).getInfo()

collection_max = processed_collection.select('LST_F').reduce(ee.Reducer.max()).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ee_santa_clara,
    scale=30,
    maxPixels=1e8,
    bestEffort=True
).getInfo()

collection_stddev = processed_collection.select('LST_F').reduce(ee.Reducer.stdDev()).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ee_santa_clara,
    scale=30,
    maxPixels=1e8,
    bestEffort=True
).getInfo()

# Print overall statistics
print("\nOverall Collection Statistics:")
print(f"Min LST: {collection_min}")
print(f"Max LST: {collection_max}")
print(f"StdDev LST: {collection_stddev}")


Overall Collection Statistics:
Min LST: {'LST_F_min': 52.80355900427356}
Max LST: {'LST_F_max': 136.98267087577327}
StdDev LST: {'LST_F_stdDev': 13.78945279132008}


Per-Image Standard Deviation: For each image, the standard deviation shows the variability of LST across the Hurley area at a specific time. A low standard deviation means the temperatures are relatively uniform across the region, while a high standard deviation suggests there are large differences in temperature within that image, perhaps due to mixed land covers or varying microclimates.

Overall Collection Standard Deviation: When calculated across the entire collection, the standard deviation reflects how much the mean LST changes over time. A low value indicates that the temperatures are fairly stable across the whole period, while a high standard deviation suggests significant temperature changes over time, such as seasonal variations or extreme weather events.

In [32]:
# Create the DataFrame
data = {
    'Date': [ee.Date(date).format("YYYY-MM-dd").getInfo() for date in dates],
    'Mean_LST': means,
    'Min_LST': mins,
    'Max_LST': maxs,
    'StdDev_LST': std_devs
}
df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])

# Overall collection statistics
overall_min = 52.80
overall_max =  136.98  
overall_std_dev = 13.79
overall_mean =  113.89

# Set up the scatter plot
plt.figure(figsize=(14, 8))
sns.scatterplot(x='Date', y='Mean_LST', data=df, color='blue', label='Mean LST')

# Add error bars for standard deviation
plt.errorbar(df['Date'], df['Mean_LST'], yerr=df['StdDev_LST'], fmt='o', ecolor='gray', alpha=0.5, capsize=3)

# Add shaded area for Min-Max range
plt.fill_between(df['Date'], df['Min_LST'], df['Max_LST'], color='blue', alpha=0.2, label='Min-Max Range')

# Add overall statistics as horizontal lines
plt.axhline(overall_min, color='green', linestyle='--', label=f'average of the Min LST: {overall_min:.2f} °F')
plt.axhline(overall_max, color='red', linestyle='--', label=f'average of the Max LST: {overall_max:.2f} °F')
plt.axhline(overall_mean, color='orange', linestyle='--', label=f'average of the Mean LST: {overall_mean:.2f} °F')
plt.axhline(overall_mean + overall_std_dev, color='purple', linestyle='--', alpha=0.5, label=f'average of the Mean ± Std Dev({overall_std_dev})')
plt.axhline(overall_mean - overall_std_dev, color='purple', linestyle='--', alpha=0.5)

# Customize the plot
plt.xlabel('Date')
plt.ylabel('LST (°F)')
plt.title('Land Surface Temperature (LST) in Hurley, NM Over Time')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

ValueError: All arrays must be of the same length

In [28]:
""" # Compute the long-term baseline (mean of all images in the collection)
baseline = processed_collection.mean()

# Function to calculate the anomaly for each image
def calculate_anomaly(image):
    anomaly = image.select('LST_F').subtract(baseline.select('LST_F'))  # Subtract baseline pixel by pixel
    return image.addBands(anomaly.rename('LST_Anomaly'))  # Add anomaly as a new band

# Map the anomaly calculation across the processed collection
anomaly_collection = processed_collection.map(calculate_anomaly)

# Visualization parameters for anomalies
anomaly_vis_params = {
    'min': -20,  # Adjust based on expected range of anomalies
    'max': 20,
    'palette': ['blue', 'white', 'red'],  # Blue for cooler, red for warmer
}

# Add the mean anomaly layer to the map
Map = geemap.Map()
Map.center_object(ee_hurley, 12)

# Visualize mean anomaly across the collection
mean_anomaly = anomaly_collection.select('LST_Anomaly').mean()
Map.addLayer(mean_anomaly, anomaly_vis_params, 'Mean Temperature Anomaly')
Map.addLayer(ee_hurley, {}, 'Hurley Boundary')
Map
 """

" # Compute the long-term baseline (mean of all images in the collection)\nbaseline = processed_collection.mean()\n\n# Function to calculate the anomaly for each image\ndef calculate_anomaly(image):\n    anomaly = image.select('LST_F').subtract(baseline.select('LST_F'))  # Subtract baseline pixel by pixel\n    return image.addBands(anomaly.rename('LST_Anomaly'))  # Add anomaly as a new band\n\n# Map the anomaly calculation across the processed collection\nanomaly_collection = processed_collection.map(calculate_anomaly)\n\n# Visualization parameters for anomalies\nanomaly_vis_params = {\n    'min': -20,  # Adjust based on expected range of anomalies\n    'max': 20,\n    'palette': ['blue', 'white', 'red'],  # Blue for cooler, red for warmer\n}\n\n# Add the mean anomaly layer to the map\nMap = geemap.Map()\nMap.center_object(ee_hurley, 12)\n\n# Visualize mean anomaly across the collection\nmean_anomaly = anomaly_collection.select('LST_Anomaly').mean()\nMap.addLayer(mean_anomaly, anomaly_

In [29]:
""" # Set up export parameters
export_task = ee.batch.Export.image.toDrive(
    image=mean_anomaly.select('LST_Anomaly'),  # Select the LST band
    description='Hurley_LST_Anomaly',
    folder='lst',  # Optional: specify a folder in Google Drive
    fileNamePrefix='Hurley_LST_Anomaly',
    region=ee_hurley,
    scale=30,
    crs='EPSG:4326',  # 'EPSG:32612',  # Use UTM Zone 12N to match the Hurley vectors
    maxPixels=1e13
)

# Start the export task
export_task.start()
print("Exporting LST_Anomaly to Google Drive...") """

' # Set up export parameters\nexport_task = ee.batch.Export.image.toDrive(\n    image=mean_anomaly.select(\'LST_Anomaly\'),  # Select the LST band\n    description=\'Hurley_LST_Anomaly\',\n    folder=\'lst\',  # Optional: specify a folder in Google Drive\n    fileNamePrefix=\'Hurley_LST_Anomaly\',\n    region=ee_hurley,\n    scale=30,\n    crs=\'EPSG:4326\',  # \'EPSG:32612\',  # Use UTM Zone 12N to match the Hurley vectors\n    maxPixels=1e13\n)\n\n# Start the export task\nexport_task.start()\nprint("Exporting LST_Anomaly to Google Drive...") '

Hurley Area

In [1]:
(1874540.995237139 + 385529.09372809314)

2260070.088965232

In [3]:
2260070.088965232/4046.85642

558.4754818074895

In [ ]:
558.4754818074895

SC AREA

In [4]:
2787514.188/4046.85642

688.8097571793763

Lordsburg Area

In [5]:
4747221.320008123/4046.85642

1173.0639358853564